# Repository Demo



Here is a flowchart for the actions that the tool takes.

Each of the following blocks are described in more detail through this notebook. Feel free to change parameters and experiment!

<p align="center">
  <img src="flowchart.drawio.png" />
</p>

## 1.1 Ensure setup is accurate

```bash
pip3 install -r pip-requirements.txt
sed 's/#.*//' apt-requirements.txt | xargs sudo apt-get install
```

In [ ]:
import os
import sys
import subprocess
import numpy as np
import pickle as pkl
from pathlib import Path
from datetime import datetime
from itertools import chain, combinations
from typing import Tuple, Dict, Any, List
from confusion_matrix import ConfusionMatrix
from generate_confusion_matrix import GenerateConfusionMatrix


from custom_env import home_dir, cm_dir, repo_dir, output_dir, preds_dir, model_dir, is_set_to_mini

from nuscenes import NuScenes
from nuscenes.eval.common.config import config_factory
from nuscenes.eval.common.data_classes import EvalBoxes

## 1.2 Setup custom_env.py

Instructions for setting up nuscenes can be found at [MMDetection3D Dataset Preperation](https://mmdetection3d.readthedocs.io/en/latest/user_guides/dataset_prepare.html)

The following cell needs to be filled out. This file is your custom environemnt and defines where things are located and whenere outputs should go. If you choose to run the tool with `verbose = True`, outputs of certain files might be placed in these locations.

In [ ]:
######## PARMS #########
## ONLY Change model_name here to make it work with your version.
model_name = "model2_good"  # The name of the directory where the ML model for inference is stored
modality = "lidar"          # The modality of the data
is_mini = True              # Are you using this on NuScenes Mini?

## Configure the name of the dataset
if is_mini:
    dataset = "nuscenes-mini"
    size = "mini"
    inf_res = "inference_results_mini"
else:
    dataset = "nuscenes-full"
    size= "full"
    inf_res = "inference_results"
    
########################
#### Get Repo Root #####
def getGitRoot():
    return subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')

def create_dir_if_not_exist(dir_path):
    if not os.path.exists(dir_path):
        print(f"Directory {dir_path} not found. Creating...")
        os.makedirs(dir_path)
    else:
        print(f"Not creating {dir_path} because it already exists")

def is_set_to_mini():
    return is_mini


home_dir = str(Path.home())
repo_dir = f"{home_dir}/nuscenes_dataset/3D_Detection"                 # The directory where the repo is stored
dataset_root = f"{home_dir}/software/mmdetection3d/data/{dataset}/"    # The directory where the dataset is stored
output_dir = f"{home_dir}/nuscenes_dataset/{inf_res}"                  # The directory where the output of inference will be stored
model_dir  = f"{output_dir}/{model_name}"                              # The directory where the inference model is stored
preds_dir  = f"{model_dir}/preds"                                      # The directory where inference predictions are stored
cm_dir = f"{repo_dir}/saved_cms/{modality}/{size}/{model_name}"        # The directory where the confusion matrices generated by the tool will be stored 
create_dir_if_not_exist(cm_dir)

## 3. Ensure NuScenes is setup correctly

In [ ]:
from custom_env import dataset_root as dataroot
from custom_env import cm_dir, model_dir, eval_version, eval_config
from custom_env import is_set_to_mini, eval_set_map, dataset_version, eval_version 

# parameters to setup nuScenes

nusc = NuScenes(version=dataset_version, dataroot = dataroot)

# Getting setup for running inference

**Config File** is a python file that contains parameters such as batch size, list of classes, indices, input size, etc.   
**Checkpoint File** is a `.pth` file which contains a the exact values of all parameters (weights, current learning rate, etc.) and stores all of this in non-volatile memory.

| Model Name | Modality |Link to Checkpoint file | Link to Config file | mAP (%) | Accuracy (%) | Link to paper |
|-|-|-|-|-|-|-|
|NuScenes SECFPN|Lidar|[Backbone file](https://download.openmmlab.com/mmdetection3d/v1.0.0_models/pointpillars/hv_pointpillars_secfpn_sbn-all_4x8_2x_nus-3d/hv_pointpillars_secfpn_sbn-all_4x8_2x_nus-3d_20210826_225857-f19d00a3.pth)|[Config File](https://github.com/open-mmlab/mmdetection3d/blob/main/configs/pointpillars/pointpillars_hv_secfpn_sbn-all_8xb4-2x_nus-3d.py)|34.33|49.1|[PointPillars](https://arxiv.org/abs/1812.05784)|
|NuScenes SECFPN(FP16)|Lidar|[Backbone file](https://github.com/open-mmlab/mmdetection3d/blob/main/configs/pointpillars/pointpillars_hv_secfpn_sbn-all_8xb2-amp-2x_nus-3d.py)|[Config file](https://download.openmmlab.com/mmdetection3d/v0.1.0_models/fp16/hv_pointpillars_secfpn_sbn-all_fp16_2x8_2x_nus-3d/hv_pointpillars_secfpn_sbn-all_fp16_2x8_2x_nus-3d_20201020_222626-c3f0483e.pth)|35.19|50.27|[PointPillars](https://arxiv.org/abs/1812.05784)|
|-|-|-|-|-|-|-|
|BEVFusion|Lidar + Camera|[Backbone file](https://github.com/open-mmlab/mmdetection3d/blob/fe25f7a51d36e3702f961e198894580d83c4387b/projects/BEVFusion/configs/bevfusion_lidar_voxel0075_second_secfpn_8xb4-cyclic-20e_nus-3d.py)|[Config file](https://download.openmmlab.com/mmdetection3d/v1.1.0_models/bevfusion/bevfusion_lidar_voxel0075_second_secfpn_8xb4-cyclic-20e_nus-3d-2628f933.pth)|69.6|64.9|[BEVFusion](https://arxiv.org/abs/2205.13542)|


In [ ]:
now = datetime.now()
configs_path = "configs/pointpillars/pointpillars_hv_fpn_sbn-all_8xb4-2x_nus-3d.py"
checkpoint_path = "checkpoints/hv_pointpillars_fpn_sbn-all_4x8_2x_nus-3d_20210826_104936-fca299c1.pth"
TAG = "xyz model run"

folder_name = "model_"+now.strftime("%m-%d-%Y_%H_%M")
out_dir = f"{output_dir}" + folder_name

if not os.path.exists(out_dir):
    os.mkdir(out_dir)

info_file = os.path.join(out_dir, "model_info.txt")
with open(info_file, 'w') as f:
    f.write(f"configs_path = {configs_path} \n checkpoint_path = {checkpoint_path} \n")
f.close()
    
pcd_path = f"{dataset_root}/samples/LIDAR_TOP/"

pcd_list = os.listdir(pcd_path)
print(len(pcd_list))

for i, pcd in enumerate(pcd_list):
    path = Path(f"{pcd_path}/{pcd}").absolute()
    if path.exists():
        cmd = f'python3 demo/pcd_demo.py {str(path)} {configs_path} {checkpoint_path} --device cuda --out-dir {out_dir}'
    
    ##### Uncomment this to run the inference #####    
    # subprocess.run(cmd, cwd=f"{home_dir}/software/mmdetection3d/", shell=True)
    
    if i%100 == 0:
        print(f"---- ---- !-!-!-!- run_inference.py: Done with {i} files")

with open(info_file, 'a') as f:
    f.write(f"Inferences complete.")
f.close()

## 4. Setup for confusion matrix generation

| Variable name | Type | Description |
|--|--|--|
| `list of classes` | `list` | The class labels for the confsion matrix |
|`conf_mat_mapping`|`dict`| Dict ***keys*** represent output classes for inference |
|`conf_mat_mapping`|`dict`| Dict ***values*** represent the class lable to match it with|
| `labels` | `dict` | Dict ***keys*** represent place in the confusion matrix |
| `labels` | `dict` | Dict ***values*** represent place in the confusion matrix   |

In [ ]:
list_of_classes = ["ped", "obs"]

PED = 0
OBS = 1
EMPTY = 2

labels = {0: "ped", 1: "obs", 2:"empty"}

conf_mat_mapping = {
    "pedestrian": PED,
    "bus": OBS,
    "car" : OBS,
    "truck": OBS,
    "bicycle": OBS,
    "motorcycle": OBS,
    "traffic_cone": OBS
}

In [ ]:
generator = GenerateConfusionMatrix(nusc=nusc,      
    config=eval_config,
    result_path=f'{model_dir}/results_nusc.json',   ## PARAM Where are the results are stored
    eval_set=eval_set_map[dataset_version],
    output_dir=os.getcwd(), #.......................## PARAM Where to store the output
    verbose=False,  #...............................## PARAM Verbose
    conf_mat_mapping=conf_mat_mapping,
    list_of_classes=list_of_classes,
    distance_parametrized=True,
    max_dist=100, #................................## PARAM The maximum distance the model considers
    distance_bin=10 #..............................## PARAM For distance parametrized confusion matrices, the distance between radius bands
)

## Setup for probability plot generation



In [ ]:
#TODO Does the env have a ped? empty? AND the user can set VMAX, Ncar 
# We generate probabilitie based on that
# cb_cm_ped

# Bibliography

```latex
@inproceedings{PointPillars,
  title={Pointpillars: Fast encoders for object detection from point clouds},
  author={Lang, Alex H and Vora, Sourabh and Caesar, Holger and Zhou, Lubing and Yang, Jiong and Beijbom, Oscar},
  booktitle={Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition},
  pages={12697--12705},
  year={2019}
}
```

```latex
@inproceedings{BevFusion,
  title={BEVFusion: Multi-Task Multi-Sensor Fusion with Unified Bird's-Eye View Representation},
  author={Liu, Zhijian and Tang, Haotian and Amini, Alexander and Yang, Xingyu and Mao, Huizi and Rus, Daniela and Han, Song},
  booktitle={IEEE International Conference on Robotics and Automation (ICRA)},
  year={2023}
}
```